In [1]:
import pandas as pd
import numpy as np

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../collection/data/profile/storage/raw/userData.csv')
bot_accounts = pd.read_csv('data/bot_id_username.csv')
clean_accounts = pd.read_csv('data/non_bot_accounts.csv')

In [3]:
bot_details = df[df['username'].isin(bot_accounts['username'])]
bot_details = bot_details.reset_index(drop=True)[:14000]

clean_details = df[df['username'].isin(clean_accounts['username'])]
clean_details = clean_details.reset_index(drop=True)

bot_details['BotOrNot'] = 1
clean_details['BotOrNot'] = 0

combined_df = clean_details.append(bot_details, ignore_index=True)

new_df = combined_df.sample(frac=1).reset_index(drop=True)
new_df['created'] = pd.to_datetime(new_df['created'])
new_df['age'] = (pd.Timestamp.today() - new_df['created']).apply(lambda x: x.days)

In [4]:
new_df = new_df[['username', 'age', 'has_location', 'is_verified', 'total_tweets', 'total_following', 'total_followers', 'total_likes', 'has_avatar', 'has_background', 'is_protected', 'profile_modified', 'BotOrNot']]

In [21]:
training_df = new_df.drop('username', axis=1)[:22000]
test_df = new_df.drop('username', axis=1)[22000:]

columns_to_standardize = ['age', 'total_tweets', 'total_following', 'total_followers', 'total_likes']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals

In [22]:
X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df.drop(['BotOrNot', 'is_protected'], axis=1).values
y_test = test_df['BotOrNot'].values.reshape(-1,1)

In [23]:
inp = Input(shape=[10])

another = Dense(500, activation='relu')(inp)
another = Dense(200, activation='relu')(another)
another = Dense(1, activation='sigmoid')(another)

mod = Model(inp, another)
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
training = mod.fit(x=X_train, y=y_train, batch_size=64, epochs=20, validation_data=(X_test, y_test))

Train on 22000 samples, validate on 5190 samples
Epoch 1/20
22000/22000 [==============================] - 4s 194us/step - loss: 0.6024 - acc: 0.6808 - val_loss: 0.5985 - val_acc: 0.6956
Epoch 2/20
22000/22000 [==============================] - 3s 144us/step - loss: 0.5854 - acc: 0.6916 - val_loss: 0.5992 - val_acc: 0.6871
Epoch 3/20
22000/22000 [==============================] - 3s 147us/step - loss: 0.5823 - acc: 0.6954 - val_loss: 0.5841 - val_acc: 0.7017
Epoch 4/20
22000/22000 [==============================] - 3s 154us/step - loss: 0.5813 - acc: 0.6966 - val_loss: 0.5860 - val_acc: 0.7042
Epoch 5/20
22000/22000 [==============================] - 3s 151us/step - loss: 0.5766 - acc: 0.6970 - val_loss: 0.5798 - val_acc: 0.7039
Epoch 6/20
22000/22000 [==============================] - 3s 140us/step - loss: 0.5739 - acc: 0.7011 - val_loss: 0.5791 - val_acc: 0.7029
Epoch 7/20
22000/22000 [==============================] - 3s 152us/step - loss: 0.5729 - acc: 0.7001 - val_loss: 0.5782 - v